In [32]:
from bert_base.client import BertClient
from elasticsearch import Elasticsearch
import time

entity_labels = ["B"]

question = "患有老年人慢性阻塞性肺疾病、新生儿呼吸暂停和先天性肺动静脉瘘这些疾病时，应该去做哪项检查？"
with BertClient(port=5555, port_out=5556, show_server_config=False, check_version=False, check_length=False,
                mode='NER') as bc:
    start_t = time.perf_counter()
    str_input_list = list(question)
    rst = bc.encode([question])
    question_labels = list(rst[0])
    print(question_labels)
    all_entity = []
    
    entity_list = []
    entity_list_number = 0
    current_b_labels = None
    for labels in question_labels:

        if labels in entity_labels:
            current_b_labels = labels

        if labels != 'O':
            entity_list.append(str_input_list[entity_list_number])
        else:
            if len(entity_list) > 0:
                entity_str = "".join(entity_list)
                if current_b_labels is not None:
                    all_entity.append((current_b_labels, entity_str))
                current_b_labels = None
                entity_list = []

        entity_list_number += 1

    if len(entity_list) > 0:
        entity_str = "".join(entity_list)
        all_entity.append((current_b_labels, entity_str))
    print(all_entity)
    ###es 内容#####
    alias = all_entity[0][1]
    es = Elasticsearch("172.17.0.7",http_auth=('elastic', 'password'), port = 9200)
    dsl = {
        "query": {
            "match": {"alias":alias }
        }
    }
    s = time.time()
    res = es.search(index="normal_index",doc_type="normal_info", size=15, body=dsl)
    print(time.time() - s)
    print("Got %d Hits:" % res['hits']['total']['value'])
    for hit in res['hits']['hits']:
        print(hit)
        
    

['O', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'I', 'I', 'I', 'I', 'I', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[('B', '老年人慢性阻塞性肺疾病'), ('B', '新生儿呼吸暂停'), ('B', '先天性肺动静脉瘘')]
0.039675235748291016
Got 10000 Hits:
{'_index': 'normal_index', '_type': 'normal_info', '_id': 'OlLVK3YBlwk3L19hhNL0', '_score': 51.26427, '_source': {'alias': '慢性阻塞性肺疾病', 'entity': '慢性阻塞性肺疾病_（医疗百科）', 'num': '2020-12-04T11:39:41.721996'}}
{'_index': 'normal_index', '_type': 'normal_info', '_id': 'swL_K3YBlwk3L19hLb0w', '_score': 47.45352, '_source': {'alias': '慢性阻塞性肺病', 'entity': '慢性阻塞性肺疾病_（医疗百科）', 'num': '2020-12-04T12:25:11.789829'}}
{'_index': 'normal_index', '_type': 'normal_info', '_id': '3uH3K3YBlwk3L19hKiS5', '_score': 46.132557, '_source': {'alias': '慢性阻塞性肺疾病控制', 'entity': '慢性阻塞性肺疾病控制', 'num': '2020-12-04T12:16:26.946142'}}
{'_index': 'normal_index', '_type': 'normal_info', '_id': 'Xm_cK3YBlwk3L19